In [5]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror{font-family:Consolas; font-size:15pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output{font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:15pt;}
div.prompt{min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_renderulli{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">ch15. 데이터베이스 연동</font></b>

# 1. SQLite 데이터베이스 연결
 - SQLite 데이터베이스는 별도의  DBMS없이  SQL문을 이용해 DB엑세스 할 수 있도록 만든 간단한 디스크 기반 DB
 - C 라이브러리
 - SQLite는 프로토타입 생성시 사용
 - 프로젝트 : 분석  ->  설계  ->  구현  ->  테스트  ->  고객에게 배포  ->  유지보수
                 프로토타입(SQLite)          완제품(Oracle, MySQL, Maria,...)
                 
- DB browswer for SQLite

## 1.1 SQLite browser 및 SQLite  패키지 load

In [8]:
import sqlite3
sqlite3.sqlite_version #버전

'3.40.1'

In [10]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터베이스 연결

    (1) 데이터베이스 연결 
    (2) SQL 전송 객체 생성 (cursor)
    (3) SQL 전송 (cursor.execute() 이용) & 결과 return
    (4) cursor 해제 & 데이터베이스 연결 객체 해제

In [28]:
#(1) DB 연결 : sqlite로 연결시 db 파일이 없으면 빈 db 파일 생성, 있으면 연결
conn = sqlite3.connect('Data/ch15_example.db')
conn

In [29]:
#(2) SQL 전송 객체 생성 (cursor 객체)
cursor = conn.cursor()
cursor

In [33]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [34]:
cursor.execute('''
    DROP TABLE MEMBER
''')

In [35]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [39]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동', 20, 'h@h.com')")
print('수행결과 행 수 : ', cursor.rowcount)

수행결과 행 수 :  1


In [37]:
sql = "INSERT INTO MEMBER VALUES ('김길동',30, 'K@K.COM')"
cursor.execute(sql)
print('수행결과 행 수 : ', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동',25, 'l@l.COM')")
print('수행결과 행 수 : ', cursor.rowcount)


수행결과 행 수 :  1
수행결과 행 수 :  1


In [40]:
conn.commit() #(반)conn.rollback()

In [42]:
#select 전송 ==> 결과 CURSOR가  가리킴
cursor.execute("SELECT * FROM MEMBER")

In [46]:
# INSERT, UPDATE, DELETE문 실행 결과 : cursor.rowcount
# SELECT문 실행 결과:
    ## fetchall() : 결과를 모두 받을 떄 (튜플 list)
    ## fetchone() : 결과를 한 행 씩 받을 떄 (튜플)
    ## fetchmany(n) : 결과를 n 행 씩 받을 떄 (list)

print(cursor.fetchall())

[]


In [47]:
print(cursor.fetchall())
#한번 소요된 cursor 객체는 다시 fetch 될 수 없음

[]


In [50]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('이길동', 25, 'l@l.COM'),
 ('김길동', 30, 'K@K.COM')]

In [54]:
#하나씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.COM')
('김길동', 30, 'K@K.COM')
데이터 끝


In [56]:
members

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('이길동', 25, 'l@l.COM'),
 ('김길동', 30, 'K@K.COM')]

In [59]:
#최상위 n 행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')


In [60]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [70]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입)'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)

def to_member(*row): # 튜플매개변수를 받아 Member형 객체를  return
    return Member(row[0],row[1],row[2])

In [72]:
member = to_member('홍길동', 20, 'h@h.com')
print(member)

홍길동	20	h@h.com


In [73]:
cursor.execute("SELECT * FROM MEMBER")
member_list = [] #sql문 수행 결과를 담을 객체 리스트체 리스트
members = cursor.fetchall() #튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [74]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
김길동	30	K@K.COM
이길동	25	l@l.COM
홍길동	20	h@h.com


In [75]:
#(4) 연결해제
cursor.close()
conn.close()

## 1.3 SQL 구문에  파라미터 사용하기
    - qmark (DB에 따라 불가능한 경우 있음)
    - named(추천)

In [76]:
conn = sqlite3.connect('Data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('홍길동', 20, 'h@h.com')]

In [81]:
#파라미터 사용하기 : qmark 방법
name1 = input('검색할이름1')
name2 = input('검색할이름2')
#cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"','"+name2+"')")
#cursor.execute(f"SELECT * FROM MEMBER WHERE NAME IN ('{name1}','{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", (name1, name2))
cursor.fetchall()

검색할이름1홍길동
검색할이름2김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('홍길동', 20, 'h@h.com')]

In [82]:
#파라미터 사용하기 : named 방법
name1 = input('검색할이름1')
name2 = input('검색할이름2')
#cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"','"+name2+"')")
#cursor.execute(f"SELECT * FROM MEMBER WHERE NAME IN ('{name1}','{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1,:name2)", {'name1':name1,
                                                                      'name2':name2})
cursor.fetchall()

검색할이름1김길동
검색할이름2이길동


[('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.COM')]

In [83]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입)'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)

def to_member(*row): # 튜플매개변수를 받아 Member형 객체를  return
    return Member(row[0],row[1],row[2])

In [92]:
#Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try:
    name = input('입력할 이름 : ')
    age  = int(input('입력할 나이 : '))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세 초기화')
    age  = 18
finally:
    email= input('입력할 메일은 : ')
inputdata = {'name':name, 'age':age, 'email':email} #named방식에서 사용
inputdata2 = (name,age,email) #qmark 방식에서 사용
sql = "INSERT INTO MEMBER VALUES (:name,:age,:email)"
cursor.execute(sql,inputdata)
conn.commit()
if cursor.rowcount:
    print('저장완료')

입력할 이름 : 마길동
입력할 나이 : a
유효하지 않은 나이를 입력한 경우, 18세 초기화
입력할 메일은 : 
저장완료


In [96]:
#Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try:
    name = input('입력할 이름 : ')
    age  = int(input('입력할 나이 : '))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세 초기화')
    age  = 18
finally:
    email= input('입력할 메일은 : ')
newMember = Member(name, age, email)
# print(newMember)
# print(newMember.__dict__)

sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, newMember.__dict__)
conn.commit()
if cursor.rowcount==1:
    print('입력성공')

입력할 이름 : 유관순
입력할 나이 : A
유효하지 않은 나이를 입력한 경우, 18세 초기화
입력할 메일은 : Y@y.com
입력성공


In [97]:
while True:
    try:
        name = input ('입력할 이름(종료는 0) :')
        if name == '0':
            break;
        age = int(input('입력할 나이 : '))
        
    except:
        print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
        age=18
    email= input('입력할 메일 :')
    newMember = Member(name, age, email)
    # print(newMember)
    # print(newMember.__dict__)

    sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
    cursor.execute(sql, newMember.__dict__)
    if cursor.rowcount==1:
        print('입력성공')
    conn.commit()

입력할 이름(종료는 0) :가나다
입력할 나이 : 20
입력할 메일 :asdf@rfr.com
입력성공
입력할 이름(종료는 0) :라마바
입력할 나이 : 40
입력할 메일 :htrt@rfs.com
입력성공
입력할 이름(종료는 0) :0


In [98]:
cursor.close()
conn.close()

# 2. 오라클 데이터베이스 연결
    -pip install cx_oracle(oracle 11g까지)
    -pip install oracledb (oracle 12부터)

In [100]:
import cx_Oracle
cx_Oracle.__version__

'8.3.0'

In [105]:
#conn 얻어오는 방법 1
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO",ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [2]:
#conn 얻어오는 방법 2
import oracledb
oracledb.init_oracle_client()
conn = oracledb.connect("scott/tiger@localhost:1521/xe")
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO",ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [5]:
#conn 얻어오는 방법 3
import oracledb
oracledb.init_oracle_client()
#conn = oracledb.connect("scott/tiger@localhost:1521/xe")
conn=oracledb.connect(
    user='scott',
    password='tiger',
    host='localhost',
    port=1521,
    sid='xe'
)
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO",ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [8]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [10]:
#select 문을 수행한 필드 정보 (NO, ENAME, JOB, ~)
cursor.description

[('NO', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, False),
 ('ENAME', <DbType DB_TYPE_VARCHAR>, 10, 10, None, None, True),
 ('JOB', <DbType DB_TYPE_VARCHAR>, 9, 9, None, None, True),
 ('MGR', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, True),
 ('HIREDATE', <DbType DB_TYPE_DATE>, 23, None, None, None, True),
 ('SAL', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('COMM', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('DEPTNO', <DbType DB_TYPE_NUMBER>, 3, None, 2, 0, True)]

In [13]:
[description[0] for description in cursor.description]
emp_df.columns = [description[0] for description in cursor.description]
emp_df.head()

,NO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [18]:
# 검색할 이름을 사용자에게 받아 해당 내용 출력
ename = input('검색할 이름은?').upper()
#print(ename)
sql = "SELECT * FROM EMP WHERE ENAME = :ename"
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
if emp:
    print(emp[0])
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름은?scott
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [21]:
fieldnames = [descrip[0] for descrip in cursor.description]
fieldnames

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [25]:
for fieldname, data in zip(fieldnames, emp[0]):
    print("{}:{}".format(fieldname,data if data is not None else ' -'))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM: -
DEPTNO:20


In [27]:
cursor.close()
conn.close()

InterfaceError: DPY-1006: cursor is not open

# 3. MySQL 데이터베이스 연결

    -pip install mysql-connector-python (공식 connector)
    -pip install pyMySQL (경량)

In [34]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [39]:
%pip uninstall mysql-connector-python

^C
Note: you may need to restart the kernel to use updated packages.


In [49]:
%pip install pyMySQL

Note: you may need to restart the kernel to use updated packages.


In [56]:
import pymysql
conn = pymysql.connect(
    host = "localhost",
    user="root",
    password="1q2w3e4r!!",
    db="devdb",
    charset="utf8mb4"
)
cursor = conn.cursor()
sql = "SELECT * FROM PERSONAL"
cursor.execute(sql)
personal = cursor.fetchall()
fieldnames = [descript[0] for descript in cursor.description]
pd.DataFrame(personal, columns=fieldnames)

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20
5,1116,johnson,manager,1001.0,1991-05-01,3550,NaN,30
6,1118,martin,analyst,1111.0,1991-09-09,3450,NaN,10
7,1121,kim,clerk,1114.0,1990-12-08,4000,NaN,20
8,1123,lee,salesman,1116.0,1991-09-23,1200,0.0,30
9,1226,park,analyst,1111.0,1990-01-03,2500,NaN,10


In [58]:
cursor.close()
conn.close()

Error: Already closed

# 4. 연습문제

## 실습형
 - 회원가입 [전체 조회]  [이름찾기] [메일삭제] [csv보내기] [종료]

## 처음 실행 

In [62]:
def load_cursor():
    global conn
    import oracledb
    oracledb.init_oracle_client()
    conn = oracledb.connect('scott/tiger@localhost:1521/xe')
    cursor = conn.cursor()
load_cursor()

1. 입력

In [166]:
def fn1_insert_member_info():
    '입력받은 데이터를 member테이블에 입력'
    cursor = conn.cursor()
    name = input('이름 : ')
    tel = input('전화 : ')
    email = input('이메일 : ')
    try:
        age = int(input('나이 : '))
        if age >200:
            age = 200
        elif age<0:
            age=0
    except:
        print('유효하지 않은 나이 입력시 나이 0으로 초기화')
        age = 0
    try:
        grade = int(input('등급 : '))
        if grade < 1:
            grade = 1
        elif grade>5:
            grade=5
    except:
        print('유효하지 않은 등급 입력시 1로 초기화')
        grade =1 
    etc = input('기타정보 : ')
    sql = "INSERT INTO MEMBER VALUES (:name, :tel, :email, :age, :grade, :etc)"
    inputdata = {'name':name, 'tel':tel, 'email':email, 'age':age, 'grade':grade, 'etc':etc}
    cursor.execute(sql, inputdata)
    conn.commit
    if cursor.rowcount==1:
        print(name,'님 가입 완료')
    cursor.close()
fn1_insert_member_info()

이름 : 가나다
전화 : 010-3534-8464
이메일 : abern@sevfl.com
나이 : 35
등급 : 3
기타정보 : na
가나다 님 가입 완료


In [170]:
def fn2_print_members():
    'member 테이블의 내용을 데이터프레임으로 display'
    import pandas as pd
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME"
    cursor.execute(sql)
    fieldnames = [description[0] for description in cursor.description]
    df = pd.DataFrame(cursor.fetchall(), columns=fieldnames)
    display(df)
    cursor.close()
fn2_print_members()

,NAME,TEL,EMAIL,AGE,GREADE,ETC
0,가나다,010-3534-8464,abern@sevfl.com,35,3,na
1,바사아,010-3653-4545,ethnbg@herc.com,53,1,없음


In [169]:
def fn3_search_member():
    '찾고자하는 이름을 input으로 받아 해당 내용을 데이터프레임으로 display'
    import pandas as pd
    cursor = conn.cursor()
    name = input('찾고자 하는 이름은 ?')
    sql = "SELECT * FROM MEMBER WHERE NAME=:name"
    cursor.execute(sql, {'name':name})
    members = cursor.fetchall()
    fieldnames = [description[0] for description in cursor.description]
    if members:
        df = pd.DataFrame(members, columns=fieldnames)
        display(df)
    else:
        print('해당 이름의 회원이 없습니다')
    cursor.close()
fn3_search_member()

찾고자 하는 이름은 ?가나다


,NAME,TEL,EMAIL,AGE,GREADE,ETC
0,가나다,010-3534-8464,abern@sevfl.com,35,3,na


In [152]:
def fn4_delete_member():
    '삭제하고자 하는 메일을 input으로 받아 해당 행을 삭제'
    cursor = conn.cursor()
    email = input('삭제하고자 하는 메일은?')
    sql = "DELETE FROM MEMBER WHERE EMAIL=:email"
    cursor.execute(sql, {'email':email})
    if cursor.rowcount:
        print(f'해당 메일({email})을 삭제하였습니다')
    else:
        print('해당 메일의 회원이 없습니다')
    cursor.close()
fn4_delete_member()

KeyboardInterrupt: Interrupted by user

In [160]:
def fn5_save_csv():
    'member 테이블 내용을 csv로 내보내기'
    import pandas as pd
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME"
    cursor.execute(sql)
    fieldnames = [description[0] for description in cursor.description]
    df = pd.DataFrame(cursor.fetchall(), columns=fieldnames)
    df.to_csv('data/ch15_member.csv', index=False)
    cursor.close()
fn5_save_csv()

In [161]:
load_cursor()
while True:
    menu = input("1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료")
    if menu=='1':
        fn1_insert_member_info()
    elif menu=='2':
        fn2_print_members()
    elif menu=='3':
        fn3_search_member()
    elif menu=='4':
        fn4_delete_member()
    elif menu=='5':
        fn5_save_csv()
    elif menu=='0':
        conn.close()
        break
    else:
        print('유효한 메뉴 번호를 입력해 주세요')

1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료1
이름 : 바사아
전화 : 010-3653-4545
이메일 : ethnbg@herc.com
나이 : 53
등급 : 1
기타정보 : 없음
바사아 님 가입 완료
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료2
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료2


KeyboardInterrupt: Interrupted by user